In [2]:
import pytorch_lightning as pl
from huggingface_hub.hf_api import HfFolder
import torch
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd

from src.failure_model import ToxicClassifier
from src.datasets import PromptOnlyDataset, SurvivalDataset
from src.conformal import conformalize
from src import utils

import time
import os
import sys
import logging


import torch._dynamo


# NOTE: supress more errors, great
torch._dynamo.config.suppress_errors = True

INFO 05-12 20:37:51 [__init__.py:239] Automatically detected platform cuda.


In [3]:
def save_results(save_path, df):
    df.to_csv(save_path, index=False)
    print(f"Results saved to {save_path}")


def load_results(save_path):
    df = pd.read_csv(save_path, index_col=None)
    print(f"Results loaded from {save_path}")
    return df

In [ ]:

TEST_PROMPTS_PATH = "data/test_prompt_only.pkl"
TEST_SURV_TIME_PATH = "data/test_surv_times.npy"

MODEL_PATH = "saved/Prop_rtp_500_ModernBERT/lightning_logs/version_3/checkpoints/epoch=0-step=99.ckpt"

TARGET_TAUS = torch.tensor([0.1])
TAUS_RANGE = torch.tensor([0.1])
TAU_TARGET_IDX = np.argmin(torch.abs(TAUS_RANGE - TARGET_TAUS)).item()

BATCH_SIZE = 1500

SAVE_PATH = "results_uncalib.csv"

In [ ]:
# load data

ds_test = PromptOnlyDataset(TEST_PROMPTS_PATH)
dl_test = DataLoader(ds_test, batch_size=1500, shuffle=False)
test_t_tilde = np.load(TEST_SURV_TIME_PATH)

print(f"Loaded {len(ds_test)} test samples.")

Loaded 12000 test samples.


In [6]:
# load model
model = ToxicClassifier.load_from_checkpoint(MODEL_PATH)
_ = model.eval()

model.set_taus(TAUS_RANGE)
model.set_min_p_for_q_tau(1e-20)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:

# NOTE: dont enable multiple-gpus for inference, as it causes weird bugs
import test


trainer = pl.Trainer(accelerator="gpu", devices=1)

# compute LPB on test set
test_pred_raw = trainer.predict(model, dataloaders=dl_test)
test_quantile_est = np.vstack([p["tau"].T for p in test_pred_raw]).clip(min=1)
tau_hat_pred = test_quantile_est[:, TAU_TARGET_IDX].flatten().astype(np.int64)
test_mean_lpb = tau_hat_pred.mean().item()

# compute LPB on test set for covered samples
covered = (test_t_tilde >= tau_hat_pred)
test_mean_covered_lpb = tau_hat_pred[covered].mean().item()

# compute miscoverage upper-bound on test set
test_miscoverage_upperbound = np.mean(test_t_tilde < tau_hat_pred)
test_miscoverage_lowerbound = np.mean(test_t_tilde < np.clip(tau_hat_pred, min=1, max=test_t_tilde.max()))

# add results to dataframe
result_dict = {
    "exp_name": "Uncalibrated",
    "exp_min_sample_size": np.nan,
    "exp_share_budget": False,
    "exp_naive": False,
    "exp_budget": np.nan,
    "exp_run_num": 0,
    "tau_hat": TARGET_TAUS[TAU_TARGET_IDX].item(),
    "max_est": np.inf,
    "cal_hours": np.nan,
    "cal_mean_generated_samples": np.nan,
    "cal_mean_c_value": np.nan,
    "cal_miscoverage": np.nan,
    "test_miscoverage_upperbound": test_miscoverage_upperbound,
    "test_miscoverage_lowerbound": test_miscoverage_lowerbound,
    "test_mean_lpb": test_mean_lpb,
    "test_mean_covered_lpb": test_mean_covered_lpb
}

print(f"Test miscoverage upperbound: {test_miscoverage_upperbound:.4f}")
print(f"Test miscoverage lowerbound: {test_miscoverage_lowerbound:.4f}")
print(f"Test mean LPB: {test_mean_lpb:.4f}")
print(f"Test mean covered LPB: {test_mean_covered_lpb:.4f}")

result_df = pd.DataFrame([result_dict])
save_results(SAVE_PATH, result_df)

/home/dahen/miniconda3/envs/llm-survival3/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dahen/miniconda3/envs/llm-survival3/lib/python ...
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/dahen/miniconda3/envs/llm-survival3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default lo

Predicting: |          | 0/? [00:00<?, ?it/s]

W0512 20:37:59.336000 3092859 site-packages/torch/_dynamo/convert_frame.py:1233] WON'T CONVERT compiled_embeddings /home/dahen/miniconda3/envs/llm-survival3/lib/python3.11/site-packages/transformers/models/modernbert/modeling_modernbert.py line 212 
W0512 20:37:59.336000 3092859 site-packages/torch/_dynamo/convert_frame.py:1233] due to: 
W0512 20:37:59.336000 3092859 site-packages/torch/_dynamo/convert_frame.py:1233] Traceback (most recent call last):
W0512 20:37:59.336000 3092859 site-packages/torch/_dynamo/convert_frame.py:1233]   File "/home/dahen/miniconda3/envs/llm-survival3/lib/python3.11/site-packages/torch/_dynamo/convert_frame.py", line 1164, in __call__
W0512 20:37:59.336000 3092859 site-packages/torch/_dynamo/convert_frame.py:1233]     result = self._inner_convert(
W0512 20:37:59.336000 3092859 site-packages/torch/_dynamo/convert_frame.py:1233]              ^^^^^^^^^^^^^^^^^^^^
W0512 20:37:59.336000 3092859 site-packages/torch/_dynamo/convert_frame.py:1233]   File "/home/dah

Test miscoverage upperbound: 0.0508
Test miscoverage lowerbound: 0.0489
Test mean LPB: 71.6433
Test mean covered LPB: 62.2507
Results saved to results_uncalib.csv


In [8]:
print(((tau_hat_pred > 2400) & (test_t_tilde == 2400)).mean())

0.0018333333333333333
